# Introduction à l’ingénierie des prompts
L’ingénierie des prompts consiste à concevoir et à optimiser des instructions pour les tâches de traitement du langage naturel. Cela implique de choisir les bons prompts, d’ajuster leurs paramètres et d’évaluer leurs performances. L’ingénierie des prompts est essentielle pour obtenir une grande précision et une efficacité optimale avec les modèles NLP. Dans cette section, nous allons découvrir les bases de l’ingénierie des prompts en utilisant les modèles OpenAI pour l’exploration.


### Exercice 1 : Tokenisation
Découvrez la tokenisation avec tiktoken, un tokenizer open-source rapide développé par OpenAI.
Consultez le [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) pour plus d'exemples.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Exercice 2 : Vérifier la configuration de la clé Github Models

Exécutez le code ci-dessous pour vérifier que votre point de terminaison Github Models est correctement configuré. Le code teste simplement une invite basique et valide la complétion. L'entrée `oh say can you see` devrait se compléter par quelque chose comme `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Exercice 3 : Fabrications
Explorez ce qui se passe lorsque vous demandez au LLM de générer des réponses à partir d’une consigne portant sur un sujet qui pourrait ne pas exister, ou sur des sujets qu’il pourrait ne pas connaître parce qu’ils sont en dehors de son jeu de données d’entraînement (plus récents). Observez comment la réponse évolue si vous essayez une consigne différente, ou un autre modèle.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Exercice 4 : Basé sur des instructions
Utilisez la variable "text" pour définir le contenu principal 
et la variable "prompt" pour donner une consigne liée à ce contenu principal.

Ici, nous demandons au modèle de résumer le texte pour un élève de CE1.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Exercice 5 : Prompt complexe
Essayez une requête qui inclut des messages système, utilisateur et assistant.
Le système définit le contexte de l’assistant.
Les messages utilisateur et assistant fournissent le contexte d’une conversation à plusieurs tours.

Remarquez comment la personnalité de l’assistant est définie comme « sarcastique » dans le contexte système.
Essayez d’utiliser un autre contexte de personnalité. Ou testez une autre série de messages d’entrée/sortie.


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Exercice : Explorez votre intuition
Les exemples ci-dessus vous donnent des modèles que vous pouvez utiliser pour créer de nouvelles invites (simples, complexes, instructions, etc.) – essayez de créer d'autres exercices pour explorer certaines des autres idées dont nous avons parlé, comme les exemples, les indices et plus encore.



---

**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction par IA [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d’origine doit être considéré comme la source faisant autorité. Pour toute information critique, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
